In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data = data.drop("date", axis=1)
data = data.fillna(0)  # Filling null values with zero
data = data.astype('float32')

# Keep data until 31.08.2023
data = data.iloc[:10731]

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Set the window size for training
train_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list = [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - train_window_size, train_window_size + 1):
    x_train_temp = x_data.iloc[i:i+train_window_size+1]
    y_train_temp = y_data.iloc[i:i+train_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_train_list.append(x_train)
    y_train_list.append(y_train)

# Convert lists to arrays
x_train_array = torch.tensor([x.values for x in x_train_list], dtype=torch.float32)
y_train_array = torch.tensor([y.values for y in y_train_list], dtype=torch.float32)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the LSTM model

num_epochs = 10000
lr=0.0001
input_size = x_train_array.shape[2]  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 1  # Number of LSTM layers
model = LSTMModel(input_size, hidden_size, num_layers)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# Define the number of epochs


# Train the model with sliding window
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i in range(len(x_train_array)):
        inputs = x_train_array[i:i+1]  # Extract one sequence at a time
        labels = y_train_array[i:i+1]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(x_train_array)}')

# Save the model if needed
torch.save(model.state_dict(), 'lstm_model.pth')


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 20])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10000], Loss: 28280.707813285335
Epoch [2/10000], Loss: 26936.757369430798
Epoch [3/10000], Loss: 25923.27606597292
Epoch [4/10000], Loss: 25026.01947859122
Epoch [5/10000], Loss: 24223.114225204212
Epoch [6/10000], Loss: 23502.56624967618
Epoch [7/10000], Loss: 22855.668926095543
Epoch [8/10000], Loss: 22275.298651004956
Epoch [9/10000], Loss: 21755.250602954053
Epoch [10/10000], Loss: 21289.95525427854
Epoch [11/10000], Loss: 20874.290595063027
Epoch [12/10000], Loss: 20503.52593763024
Epoch [13/10000], Loss: 20173.27700863239
Epoch [14/10000], Loss: 19879.475524661593
Epoch [15/10000], Loss: 19618.365529256323
Epoch [16/10000], Loss: 19386.485506040943
Epoch [17/10000], Loss: 19180.66730200381
Epoch [18/10000], Loss: 18998.024941504118
Epoch [19/10000], Loss: 18835.945418333587
Epoch [20/10000], Loss: 18692.056264694424
Epoch [21/10000], Loss: 18564.241539684528
Epoch [22/10000], Loss: 18450.600340035096
Epoch [23/10000], Loss: 18349.434315043185
Epoch [24/10000], Loss: 182

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data = data.drop("date", axis=1)
data = data.fillna(0)  # Filling null values with zero
data = data.astype('float32')

# Keep data until 31.08.2023
data = data.iloc[:10731]

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Set the window size for training
train_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list = [], []

# Iterate through the data with the specified window size
for i in range(len(x_data) - train_window_size):
    x_train_temp = x_data.iloc[i:i+train_window_size]
    y_train_temp = y_data.iloc[i+train_window_size]

    x_train_list.append(x_train_temp)
    y_train_list.append(y_train_temp)

# Convert lists to arrays
x_train_array = torch.tensor([x.values for x in x_train_list], dtype=torch.float32)
y_train_array = torch.tensor([y for y in y_train_list], dtype=torch.float32).view(-1, 1)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the LSTM model
input_size = x_train_array.shape[2]  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 1  # Number of LSTM layers
model = LSTMModel(input_size, hidden_size, num_layers)
lr=0.0001

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# Define the number of epochs
num_epochs = 20

# Train the model with sliding window
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i in range(len(x_train_array)):
        inputs = x_train_array[i:i+1]  # Extract one sequence at a time
        labels = y_train_array[i:i+1]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(x_train_array)}')

# Save the model if needed
torch.save(model.state_dict(), 'lstm_model.pth')


Epoch [1/20], Loss: 16722.34287054072
Epoch [2/20], Loss: 16737.677274132937
Epoch [3/20], Loss: 16222.152742441875
Epoch [4/20], Loss: 15733.915141268038
Epoch [5/20], Loss: 15331.599266647547
Epoch [6/20], Loss: 15008.305573436586
Epoch [7/20], Loss: 14748.260000470373
Epoch [8/20], Loss: 14536.683856590562
Epoch [9/20], Loss: 14361.582862653253
Epoch [10/20], Loss: 14213.86600263427
Epoch [11/20], Loss: 14086.787944085232
Epoch [12/20], Loss: 13975.528246190099
Epoch [13/20], Loss: 13876.582430434415
Epoch [14/20], Loss: 13787.442268329958
Epoch [15/20], Loss: 13706.28046026463
Epoch [16/20], Loss: 13631.755623140909
Epoch [17/20], Loss: 13562.843525686692
Epoch [18/20], Loss: 13498.7658754308
Epoch [19/20], Loss: 13438.918239087014
Epoch [20/20], Loss: 13382.804446322769
